<a href="https://colab.research.google.com/github/PrernaNaik23/FeedbackReviewerSys/blob/main/feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax


reviews_path = '/content/Product_Reviews_Final.csv'
reviews = pd.read_csv(reviews_path)
reviews['Reviews_Text'] = reviews['Reviews_Text'].str.lower()
stop_words = set(stopwords.words('english'))
reviews['Reviews_Text'] = reviews['Reviews_Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
punctuation_pattern = r'[^\w\s]'
text = reviews['Reviews_Text']
reviews.drop([3851, 3959], inplace=True)
reviews['Reviews_Text'] = reviews['Reviews_Text'].apply(lambda x: re.sub(punctuation_pattern, '', x))
reviews['Reviews_Text'] = reviews['Reviews_Text'].str.replace('\d+', '')
reviews = reviews[['Reviews_Text','Reviews_Rating']]
reviews['Reviews_Text'].dropna()
reviews['Reviews_Text'].reset_index(drop=True)
reviews = reviews[reviews['Reviews_Rating']!=3]
reviews['label'] = reviews['Reviews_Rating'].apply(lambda x: 1 if x>=4 else 0)
reviews['Reviews_Rating'].value_counts()
def contractions(s):
    s = re.sub(r"won't", "will not", s)
    s = re.sub(r"wouldn't", "would not", s)
    s = re.sub(r"couldn't", "could not", s)
    s = re.sub(r"can't", "can not", s)
    s = re.sub(r"n't", " not", s)
    s = re.sub(r"'re", " are", s)
    s = re.sub(r"'s", " is", s)
    s = re.sub(r"'ll", " will", s)
    s = re.sub(r"'ve", " have", s)
    s = re.sub(r"'m", " am", s)
    s = re.sub(r"'d", " would", s)
    return s
reviews['Reviews_Text'] = reviews['Reviews_Text'].apply(contractions)
lemma = WordNetLemmatizer()
reviews['Reviews_Text'] = reviews['Reviews_Text'].apply(lambda x: ' '.join([lemma.lemmatize(word) for word in nltk.word_tokenize(x)]))
model = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architec

In [33]:
from tqdm import tqdm

def polarity_scores_roberta(text):
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict
res = {}
for i, row in tqdm(reviews.iterrows(), total=len(reviews)):
  try:
    text = row['Reviews_Text']
    roberta_result = polarity_scores_roberta(text)
    res[i] = roberta_result
  except RuntimeError:
    print(f'Broke for id {i}')
polarity_df = pd.DataFrame(res).T
reviews = reviews.merge(polarity_df, left_index=True, right_index=True)
print(reviews.head())

100%|██████████| 4801/4801 [11:51<00:00,  6.74it/s]


                                        Reviews_Text  Reviews_Rating  label  \
0  older version kindle ereader allowed download ...               5      1   
1                   far good wife happy miss old one               5      1   
2  excellent ereader looking for probably bit pri...               4      1   
3  old kindle touch last one still texttovoice lo...               4      1   
4  kindle light easy hold long time without getti...               5      1   

   roberta_neg  roberta_neu  roberta_pos  
0     0.339945     0.598606     0.061448  
1     0.008976     0.033567     0.957457  
2     0.009584     0.084028     0.906389  
3     0.088369     0.434062     0.477569  
4     0.014795     0.449097     0.536108  
